## Car Data

* Ekte 150 bin ile 220 bin lira arasında fiyatlanmış 2. el arabaların datası verilmektedir.

1. Fiyati en cok belirleyen parametreler nelerdir? Bu parametreleri kullanarak fiyati tahmin edecek bir model için öneriniz nedir? Bu modelin performansi hakkinda ne dersiniz?
    * Lineer regression, r^2 -> 1
    * 
2. Fiyati ucuz ve pahali olmak uzere 2 kategoriye ayirdigimizi farz edelim, basit bir binary classifier (mesela logistic regression) kullanarak modelinizin performansini nasil olcumlersiniz?

3. Fiyati 2 kategori yerine 3 kategoriye ayırarak (multi-nominal, ordinal) veri oluşturduğumuzda yukarıdaki modele göre performansta nasil bir degisiklik oluyor. Degisik kategorilerin cutoff pointlerini optimal olarak nasil belirleriz?
    Yukarıda önerdiğiniz 3 değişik modeli roc ve auc olarak karşılaştırmak gerekirse hangisini ve hangi kategoriler icin tercih edersiniz?
    
4. Araba almak isteyen birine bu listedeki arabalara bakarak nasil bir karar verme mekanizmasi kullanarak araba almasını önerirsiniz? Bu soruya cevap verirken yukarda kullandiginiz modelleri ve elde edilen içgörü ve öngörüleri kullanabilirsiniz. Verilerin yaklaşık 2 sene önceye ait olduğunu ayrıca belirtmek isterim.

In [1]:
# load modules

import pandas as pd
import numpy as np
import collections
import itertools
import matplotlib.pyplot as plt
from operator import itemgetter

In [2]:
df = pd.read_csv("data.csv")

In [ ]:
df.info()

In [3]:
df.head(21)

,Sehir,Marka,Seri,Model,Yil,Yakit,Km,Hp,Cc,Fiyat,Sehirici,Sehirdisi,Ortalama,Bagaj
0,Tekirdag,Dacia,0.9 TCe Stepway Easy-R,Sandero,2020,Benzin,3000,90.0,898.0,152500,5.7,4.7,5.1,320.0
1,Kocaeli,Hyundai,1.6 CRDI Mode Plus,Accent Blue,2018,Dizel,48000,136.0,1582.0,152500,5.7,4.0,4.6,465.0
2,Mugla,Nissan,1.2 Match,Micra,2016,Benzin,40000,80.0,1198.0,152500,6.7,4.6,5.4,265.0
3,Eskisehir,Nissan,1.2 Match,Micra,2017,Benzin,35000,80.0,1198.0,152250,6.7,4.6,5.4,265.0
4,Istanbul,Renault,1.2 Turbo Icon,Clio,2015,Benzin,37800,120.0,1197.0,152500,6.6,4.4,5.2,300.0
5,Istanbul,Nissan,1.2 Street,Micra,2017,Benzin,36000,80.0,1198.0,152500,6.7,4.6,5.4,265.0
6,Kocaeli,Citroen,1.6 VTi Feel,C3,2018,Benzin,18800,115.0,1587.0,152500,9.2,5.3,6.8,300.0
7,Istanbul,Skoda,1.0 GreenTec Style,Fabia,2017,Benzin,19000,110.0,999.0,152500,5.4,4.1,4.6,305.0
8,Edirne,Peugeot,Active,208 1.2 PureTech,2018,Benzin,41250,82.0,1199.0,152500,4.6,3.9,4.2,285.0
9,Adana,Dacia,0.9 TCe Stepway Easy-R,Sandero,2020,Benzin,0,90.0,898.0,152250,5.7,4.7,5.1,320.0


In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
list(df.columns)

### Arabaları Şehirlere Gruplayıp Fiyat Ortalamalarına Göre Numerik Label Ver

In [ ]:
cities = sorted(list(set(df["Sehir"])))

car_counts = list(df.groupby(["Sehir"])["Fiyat"].count())

In [ ]:
cars_by_cities = sorted(zip(cities, car_counts), key=itemgetter(1))

# Nan Verileri Değiştir

#### Sehirici
* Şehiriçi yakıt sütununun nan değerlerini değiştirirken her bir şehrin ortalamasını al.

In [ ]:
df_empty_Sehirici = df[df.Sehirici.isnull()]
cities = sorted(list(set(df_empty_Sehirici["Sehir"])))

In [ ]:
values = dict(df.groupby("Sehir")["Sehirici"].mean())

In [ ]:
nulls = list(df[df["Sehirici"].isnull()].index)

for i in nulls:
    df.iloc[i,10] = values[df.iloc[i,0]]

#### Sehirdisi
* Şehirdışı için yakıt sütununu değiştirirken tüm ortalamaya göre nan verileri doldur.

In [ ]:
df_by_Sehirdisi = np.float64(df["Sehirdisi"].mode())
df.Sehirdisi = df.Sehirdisi.fillna(value=df_by_Sehirdisi)

## Bagaj

In [5]:
# Boş bagajları tespit et

nulls = list(df[df["Bagaj"].isnull()].index)

458

In [9]:
for i in df[df["Bagaj"].isnull()].groupby(["Marka","Seri","Model"]):
    print(i)

(('Audi ', 'A3 '),          Sehir  Marka                             Seri Model   Yil   Yakit  \
3234  Istanbul  Audi   A3 Hatchback 1.4 TFSI 1.4 TFSI    A3   2009  Benzin   

         Km     Hp      Cc   Fiyat  Sehirici  Sehirdisi  Ortalama  Bagaj  
3234  40000  113.0  1450.5  180000       NaN        NaN       NaN    NaN  )
(('Citroen ', 'C3 1.2 PureTech '),           Sehir     Marka               Seri             Model   Yil   Yakit  \
3057    Antalya  Citroen               Feel   C3 1.2 PureTech   2019  Benzin   
3391   Istanbul  Citroen   Shine SX Edition   C3 1.2 PureTech   2020  Benzin   
3423     Ankara  Citroen               Feel   C3 1.2 PureTech   2020  Benzin   
3457     Samsun  Citroen          Feel Bold   C3 1.2 PureTech   2020  Benzin   
3519      Bursa  Citroen          Feel Bold   C3 1.2 PureTech   2020  Benzin   
3777    Antalya  Citroen             Shine    C3 1.2 PureTech   2019  Benzin   
3814   Istanbul  Citroen          Feel Bold   C3 1.2 PureTech   2020  Benzin  

In [ ]:
cars_by_bagaj = df.groupby(["Marka","Model"]).count()

dict(cars_by_bagaj.Sehirici)